## Writing Functions

The goal of this exercise is to replicate the process from the slides, but with multivariate data. You should think of this as a practise of two things: 

1. Performing linear algebra in R. This is straight forward, but takes some practise. Especially worth noting how to use vectors and scalars together, as you see how to modify the functions we originally wrote for scalars, such that they work for vectors.

2. Writing small functions and testing them. Make sure each part works before trying to combine them into a whole!

In [ ]:
library(tidyverse)

In [2]:
# Write a function that generates data for regressions, 

# y should be generated according to: 
# y <- X %*% beta + eps
# X should be several columns of independent random normals (aka a multivariate random 
# normal matrix with diagonal covariance)
# eps should be such that:
# eps <- rnorm(1, 0, sd)

# beta should be a vector
# params should be a DATAFRAME/TIBBLE with two columns: "mean" and "sd" to generate the features (X) 
# sd should be a scalar: the standard deviation of the normally distributed error term in the DGP

# hint: use cbind to bind vectors into the columns of a matrix or the library MASS with the function mvrnorm


generate_data <- function(N, beta, sd) { 
  X <- rnorm(N, , 1)
  eps <- rnorm(N, 0, sd)
  y <- beta*X + eps
  list(X=X, y=y)
}


$X
 [1]  0.06374446  1.05051322  0.76279928  1.22849112  0.74397985 -0.09830338
 [7]  1.54186516 -1.87502449  0.31135102  0.31577730

$y
 [1]  0.1274889  2.1010264  1.5255986  2.4569822  1.4879597 -0.1966068
 [7]  3.0837303 -3.7500490  0.6227020  0.6315546

In [326]:
# Write other helper functions, as in the slides, and eventually a function "avg_simulations" that takes
# the same parameters as generate_data, plus an "M" parameter that controls how many simulations are run

## Tests

Make sure the following assertions pass:

In [ ]:
a <- avg_simulations(1000, 
                20, 
                c(1,2,1), 
                data.frame(list(mean=c(0,0,0), sd=c(.2, .5, .3))), 
                .5)

stopifnot(round(a, 1) == .9)

In [ ]:
a <- avg_simulations(1000, 
                500, 
                c(1,2,1), 
                data.frame(list(mean=c(0,0,0), sd=c(.2, .5, .3))), 
                .5)

stopifnot(a > .92)